<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>


<h1 align="right">KSO Tutorials #1: Upload new footage</h1>
<h3 align="right">Written by @jannesgg and @vykanton</h3>
<h5 align="right">Last updated: Mar 15th, 2022</h5>

# Set up and requirements

Import Python packages

In [ ]:
# Set the directory of the libraries
import sys
sys.path.append('..')

# Set to display dataframes as interactive tables
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)

# Import required modules
import kso_utils.tutorials_utils as t_utils
import kso_utils.t1_utils as t1
import kso_utils.project_utils as p_utils
#import utils.server_utils as serv_utils

print("Packages loaded successfully")

### Choose your project

In [ ]:
project_name = t_utils.choose_project()

In [ ]:
project = p_utils.find_project(project_name=project_name.value)

### Initiate sql and get server or local storage details

In [ ]:
# Initiate db
db_info_dict = t_utils.initiate_db(project)

# Select the survey linked to the deployment

In [ ]:
survey_i = t1.select_survey(db_info_dict)

In [ ]:
survey_name = t1.confirm_survey(survey_i, db_info_dict)

## Select the deployment of interest

In [ ]:
import os
from ipyfilechooser import FileChooser
from ipywidgets import interactive, Layout, Button, HBox 

import asyncio
import kso_utils.movie_utils as movie_utils
import kso_utils.server_utils as server_utils
import pandas as pd
import ipywidgets as widgets
import numpy as np
import subprocess
import datetime
import logging
from tqdm import tqdm

In [ ]:
# Select site and date of the video
def select_SiteID(db_initial_info):
    
    # Read csv as pd
    sitesdf = pd.read_csv(db_initial_info["local_sites_csv"])

    # Existing sites
    exisiting_sites = sitesdf.sort_values("SiteID").SiteID.unique()
    
    site_widget = widgets.Dropdown(
                options = exisiting_sites,
                description = 'Site ID:',
                disabled = False,
                layout=Layout(width='50%'),
                style = {'description_width': 'initial'}
            )
#     display(site_widget)

    return site_widget


def select_eventdate(default_date):
    # Select the date 
    date_widget = widgets.DatePicker(
        description='Deployment or event date:',
        value = default_date,
        disabled=False,
        layout=Layout(width='50%'),
        style = {'description_width': 'initial'}
    )
#     display(date_widget)
    
    return date_widget  

In [ ]:
def select_date_site(db_info_dict, survey_i):
    
    ###### Save the name of the survey
    # Load the csv with with sites and survey choices
    choices_df = pd.read_csv(db_info_dict["local_choices_csv"])
    
    if isinstance(survey_i.result, dict):
        # Save the responses as a new row for the survey csv file
        new_survey_row_dict = {key: (value.value if hasattr(value, 'value') else value.result if isinstance(value.result, int) else value.result.value) for key, value in survey_i.result.items()}
        new_survey_row = pd.DataFrame.from_records(new_survey_row_dict, index=[0])

        # Save the year of the survey
#         survey_year = new_survey_row["SurveyStartDate"].values[0].strftime("%Y")
        survey_year = new_survey_row["SurveyStartDate"].dt.year.values[0]
        
                
    else:
        # Read surveys csv
        surveys_df = pd.read_csv(db_info_dict["local_surveys_csv"],parse_dates=['SurveyStartDate'])
        
        # Return the name of the survey
        survey_name = survey_i.result.value
        
        # Save the SurveyID that match the survey name
        new_survey_row = surveys_df[surveys_df["SurveyName"]==survey_name].reset_index(drop=True)
                
        # Save the year of the survey
        survey_year = new_survey_row["SurveyStartDate"].dt.year.values[0]
                       
        
    # Get prepopulated fields for the survey
    new_survey_row[["ShortFolder"]] = choices_df[choices_df["MarineReserve"]==new_survey_row.LinkToMarineReserve.values[0]][["ShortFolder"]].values[0]

    # Save the "server filename" of the survey 
    short_marine_reserve = new_survey_row["ShortFolder"].values[0]

    # Save the "server filename" of the survey 
    survey_server_name = short_marine_reserve + "-buv-" + str(survey_year) + "/"
            
    # Save the beginning of the survey
    survey_start = pd.Timestamp(new_survey_row["SurveyStartDate"].values[0]).to_pydatetime()
        
    # Retrieve deployments info from the survey of interest
    deployments_server_name = db_info_dict["client"].list_objects(Bucket=db_info_dict["bucket"],
                                                            Prefix=survey_server_name,
                                                            Delimiter='/')

    # Convert info to dataframe
    deployments_server_name = pd.DataFrame.from_dict(deployments_server_name['CommonPrefixes'])
                
    # Define function to select folder with go-pro files
    def deployment_exist(site, date):
        # Save eventdate as str
        date_str = date.strftime("%d_%m_%Y")

        # Specify the name of the deployment
        deployment_date_site = site+"_"+date_str
    
        print("Looking for ", deployment_date_site)
        print(deployments_server_name['Prefix'].to_list())
        
        # If deployment_date_site selected exists...
        if deployment_date_site in deployments_server_name['Prefix'].to_list():
            
            # Widget to verify the video is correct
            w1 = interactive(sel_subfolder,
                             survey_folder = widgets.Dropdown(
                                 options = deployments_server_name['Prefix'].to_list(),
                                 description = 'Select the folder of the survey to process:',
                                 disabled = False,
                                 layout=Layout(width='90%'),
                                 style = {'description_width': 'initial'}
                             )
                            )
#             display(w1)

            return w1
            
            
        else:
            print(deployment_date_site, "is not in the server.")
            
            # Widget to select the local folder of interest
            fc = FileChooser('/')
            fc.title = '<b>Select the local folder with the deployment videos</b>'
            # Switch to folder-only mode
            fc.show_only_dirs = True
            display(fc)

            return fc
    
    
    # Display the options
    w = interactive(deployment_exist,
                    site = select_SiteID(db_info_dict),
                    date = select_eventdate(survey_start),
                   )

#     display(w)

    return w

#datetime.date(2020,1,1)

In [ ]:
select_date_site(db_info_dict, survey_i)

# Select the folder with the go pro files of the deployment

Make sure you select the FOLDER not the files

In [ ]:
go_pro_folder = t1.select_go_pro_folder(db_info_dict)

In [ ]:
go_pro_files = t1.select_go_pro_files(go_pro_folder, db_info_dict)

## Concatenate the go pro files

In [ ]:
video_info_dict = t1.concatenate_go_pro_videos(db_info_dict,
                                                SiteID_i.value,
                                                EventDate_i.value,
                                                go_pro_folder,
                                                go_pro_files)

## Specify deployment details

In [ ]:
deployment_info = t1.record_deployment_info(db_info_dict, video_info_dict)

## Review deployment  details

In [ ]:
new_deployment_row = t1.confirm_deployment_details(video_info_dict,
                                                   db_info_dict,
                                                   survey_i,
                                                   deployment_info)

# !!!Only pass this point if deployment details are correct!!!

### Update movies csv and upload video to s3

In [ ]:
t1.upload_concat_movie(db_info_dict, video_info_dict, new_deployment_row)

In [ ]:
#END